# 数据处理

In [39]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('US-pumpkins.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1757 entries, 0 to 1756
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City Name        1757 non-null   object 
 1   Type             45 non-null     object 
 2   Package          1757 non-null   object 
 3   Variety          1752 non-null   object 
 4   Sub Variety      296 non-null    object 
 5   Grade            0 non-null      float64
 6   Date             1757 non-null   object 
 7   Low Price        1757 non-null   float64
 8   High Price       1757 non-null   float64
 9   Mostly Low       1654 non-null   float64
 10  Mostly High      1654 non-null   float64
 11  Origin           1754 non-null   object 
 12  Origin District  131 non-null    object 
 13  Item Size        1478 non-null   object 
 14  Color            1141 non-null   object 
 15  Environment      0 non-null      float64
 16  Unit of Sale     162 non-null    object 
 17  Quality       

,City Name,Type,Package,Variety,Sub Variety,Grade,Date,Low Price,High Price,Mostly Low,...,Unit of Sale,Quality,Condition,Appearance,Storage,Crop,Repack,Trans Mode,Unnamed: 24,Unnamed: 25
0,BALTIMORE,NaN,24 inch bins,NaN,NaN,NaN,4/29/17,270.0,280.0,270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,E,NaN,NaN,NaN
1,BALTIMORE,NaN,24 inch bins,NaN,NaN,NaN,5/6/17,270.0,280.0,270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,E,NaN,NaN,NaN
2,BALTIMORE,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,9/24/16,160.0,160.0,160.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
3,BALTIMORE,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,9/24/16,160.0,160.0,160.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
4,BALTIMORE,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,11/5/16,90.0,100.0,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN


In [40]:
df.shape

(1757, 26)

In [41]:
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

Type               1712
Variety               5
Sub Variety        1461
Grade              1757
Mostly Low          103
Mostly High         103
Origin                3
Origin District    1626
Item Size           279
Color               616
Environment        1757
Unit of Sale       1595
Quality            1757
Condition          1757
Appearance         1757
Storage            1757
Crop               1757
Trans Mode         1757
Unnamed: 24        1757
Unnamed: 25        1654
dtype: int64


In [42]:
# 定义缺失值比例阈值
missing_threshold = 0.75

# 删除缺失值比例超过阈值的列
drop = df.columns[df.isnull().mean() > missing_threshold]
df = df.drop(drop, axis=1)

missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

Variety          5
Mostly Low     103
Mostly High    103
Origin           3
Item Size      279
Color          616
dtype: int64


In [43]:
# 填充缺失值
df['Variety'] = df['Variety'].fillna('unknown')
df['Origin'] = df['Origin'].fillna('unknown')
df['Item Size'] = df['Item Size'].fillna('unknown')
df['Color'] = df['Color'].fillna('unknown')
df['Mostly High'] = df['Mostly High'].fillna(df['Mostly High'].mean())
df['Mostly Low'] = df['Mostly Low'].fillna(df['Mostly Low'].mean())

In [44]:
df.isnull().sum()

City Name      0
Package        0
Variety        0
Date           0
Low Price      0
High Price     0
Mostly Low     0
Mostly High    0
Origin         0
Item Size      0
Color          0
Repack         0
dtype: int64

In [45]:
# 将价格列转换为数值类型（去除美元符号并转换为浮点数）
price_columns = ['Low Price', 'High Price', 'Mostly Low', 'Mostly High']
for col in price_columns:
    df[col] = df[col].replace({'\$': '', ',': ''}, regex=True).astype(float)

# 将日期列转换为日期类型
df['Date'] = pd.to_datetime(df['Date'])

In [46]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1757 entries, 0 to 1756
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   City Name    1757 non-null   object        
 1   Package      1757 non-null   object        
 2   Variety      1757 non-null   object        
 3   Date         1757 non-null   datetime64[ns]
 4   Low Price    1757 non-null   float64       
 5   High Price   1757 non-null   float64       
 6   Mostly Low   1757 non-null   float64       
 7   Mostly High  1757 non-null   float64       
 8   Origin       1757 non-null   object        
 9   Item Size    1757 non-null   object        
 10  Color        1757 non-null   object        
 11  Repack       1757 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(7)
memory usage: 164.8+ KB


,City Name,Package,Variety,Date,Low Price,High Price,Mostly Low,Mostly High,Origin,Item Size,Color,Repack
0,BALTIMORE,24 inch bins,unknown,2017-04-29,270.0,280.0,270.0,280.0,MARYLAND,lge,unknown,E
1,BALTIMORE,24 inch bins,unknown,2017-05-06,270.0,280.0,270.0,280.0,MARYLAND,lge,unknown,E
2,BALTIMORE,24 inch bins,HOWDEN TYPE,2016-09-24,160.0,160.0,160.0,160.0,DELAWARE,med,ORANGE,N
3,BALTIMORE,24 inch bins,HOWDEN TYPE,2016-09-24,160.0,160.0,160.0,160.0,VIRGINIA,med,ORANGE,N
4,BALTIMORE,24 inch bins,HOWDEN TYPE,2016-11-05,90.0,100.0,90.0,100.0,MARYLAND,lge,ORANGE,N


In [48]:
# 识别分类变量列
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print("分类变量列：", categorical_cols)

# 对分类变量进行独热编码
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
print("\n独热编码后数据集形状：", df_encoded.shape)

# 查看编码后数据示例
print("\n独热编码后数据前5行：")
pd.DataFrame(df_encoded.iloc[:5, :]).head()

分类变量列： ['City Name', 'Package', 'Variety', 'Origin', 'Item Size', 'Color', 'Repack']

独热编码后数据集形状： (1757, 76)

独热编码后数据前5行：


,Date,Low Price,High Price,Mostly Low,Mostly High,City Name_BALTIMORE,City Name_BOSTON,City Name_CHICAGO,City Name_COLUMBIA,City Name_DALLAS,...,Item Size_lge,Item Size_med,Item Size_med-lge,Item Size_sml,Item Size_unknown,Item Size_xlge,Color_STRIPED,Color_WHITE,Color_unknown,Repack_N
0,2017-04-29,270.0,280.0,270.0,280.0,True,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
1,2017-05-06,270.0,280.0,270.0,280.0,True,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,2016-09-24,160.0,160.0,160.0,160.0,True,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
3,2016-09-24,160.0,160.0,160.0,160.0,True,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
4,2016-11-05,90.0,100.0,90.0,100.0,True,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
